In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')
import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')
import gspread

In [2]:
df = pd.read_csv('wfp_food_prices_sdn.csv')
df.head()

,date,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice
0,#date,#adm1+name,#adm2+name,#loc+market+name,#geo+lat,#geo+lon,#item+type,#item+name,#item+unit,#item+price+flag,#item+price+type,#currency,#value,#value+usd
1,2001-01-15,Blue Nile,Damazin,Damazin,11.7891,34.3592,cereals and tubers,Sorghum (white),90 KG,actual,Wholesale,SDG,46.0,17.8728
2,2001-01-15,Gedaref,Al Gedaref Rural,El Gedarif,14.04,35.38,cereals and tubers,Sorghum (white),90 KG,actual,Wholesale,SDG,51.67,20.0758
3,2001-01-15,Khartoum,Jebal Aulya,Khartoum,15.5126,32.5436,cereals and tubers,Millet,90 KG,actual,Wholesale,SDG,77.29,30.0302
4,2001-01-15,Khartoum,Jebal Aulya,Khartoum,15.5126,32.5436,cereals and tubers,Sorghum (white),90 KG,actual,Wholesale,SDG,63.9,24.8277


# Data Preprocessing

In [3]:
df.columns

Index(['date', 'admin1', 'admin2', 'market', 'latitude', 'longitude',
       'category', 'commodity', 'unit', 'priceflag', 'pricetype', 'currency',
       'price', 'usdprice'],
      dtype='object')

In [4]:
# Rename columns and remove non-food data 

new_column_names = {
    'admin1': 'state',
    'admin2': 'settlement',
}
df.rename(columns=new_column_names, inplace=True)
df = df.drop(0)
df = df[df['category'] != 'non-food']
df['unit'] = df['unit'].str.replace(' KG', '').astype(float)
df.rename(columns={'unit': 'unit(KG)'}, inplace=True)
df.head()

,date,state,settlement,market,latitude,longitude,category,commodity,unit(KG),priceflag,pricetype,currency,price,usdprice
1,2001-01-15,Blue Nile,Damazin,Damazin,11.7891,34.3592,cereals and tubers,Sorghum (white),90.0,actual,Wholesale,SDG,46.0,17.8728
2,2001-01-15,Gedaref,Al Gedaref Rural,El Gedarif,14.04,35.38,cereals and tubers,Sorghum (white),90.0,actual,Wholesale,SDG,51.67,20.0758
3,2001-01-15,Khartoum,Jebal Aulya,Khartoum,15.5126,32.5436,cereals and tubers,Millet,90.0,actual,Wholesale,SDG,77.29,30.0302
4,2001-01-15,Khartoum,Jebal Aulya,Khartoum,15.5126,32.5436,cereals and tubers,Sorghum (white),90.0,actual,Wholesale,SDG,63.9,24.8277
5,2001-01-15,Khartoum,Jebal Aulya,Khartoum,15.5126,32.5436,cereals and tubers,Wheat,90.0,actual,Wholesale,SDG,81.5,31.666


In [5]:
#Convert object types
df['date'] = pd.to_datetime(df['date'])
df['price'] = df['price'].astype(float)
df['usdprice'] = df['usdprice'].astype(float)

# Create new price per KG column 
df['price per KG(SDG)'] = df['price']/ df['unit(KG)']
df['price per KG(USD)'] = df['usdprice']/ df['unit(KG)']

# Create new year column 
df['year'] = df['date'].dt.year

df['year'] = df['year'].astype(int)
df.head()
# Create new region column 
region_mapping = {
    'Khartoum': 'Central', 'North Kordofan': 'Central', 'Al Gezira': 'Central',
    'Northern': 'North', 'Nile': 'North',
    'South Kordofan': 'South', 'Blue Nile': 'South', 'White Nile': 'South',
    'Eastern Darfur': 'South', 'West Kordofan': 'South',
    'Kassala': 'East', 'Red Sea': 'East', 'Gedaref': 'East', 'Sennar': 'East',
    'North Darfur': 'West', 'South Darfur': 'West',
    'West Darfur': 'West', 'Central Darfur': 'West'
}


df['region'] = df['state'].map(region_mapping)
df.head()

,date,state,settlement,market,latitude,longitude,category,commodity,unit(KG),priceflag,pricetype,currency,price,usdprice,price per KG(SDG),price per KG(USD),year,region
1,2001-01-15,Blue Nile,Damazin,Damazin,11.7891,34.3592,cereals and tubers,Sorghum (white),90.0,actual,Wholesale,SDG,46.00,17.8728,0.511111,0.198587,2001,South
2,2001-01-15,Gedaref,Al Gedaref Rural,El Gedarif,14.04,35.38,cereals and tubers,Sorghum (white),90.0,actual,Wholesale,SDG,51.67,20.0758,0.574111,0.223064,2001,East
3,2001-01-15,Khartoum,Jebal Aulya,Khartoum,15.5126,32.5436,cereals and tubers,Millet,90.0,actual,Wholesale,SDG,77.29,30.0302,0.858778,0.333669,2001,Central
4,2001-01-15,Khartoum,Jebal Aulya,Khartoum,15.5126,32.5436,cereals and tubers,Sorghum (white),90.0,actual,Wholesale,SDG,63.90,24.8277,0.710000,0.275863,2001,Central
5,2001-01-15,Khartoum,Jebal Aulya,Khartoum,15.5126,32.5436,cereals and tubers,Wheat,90.0,actual,Wholesale,SDG,81.50,31.6660,0.905556,0.351844,2001,Central


In [6]:
# Make FataFrame Pipeline Interactive 
idf = df.interactive()

# (1) Food prices over time by region 

In [7]:
# Define Panel widgets 

import panel as pn

# Extract years from the 'Date' column
# years = sorted(df['date'].dt.year.unique())
year_slider = pn.widgets.IntSlider(name='Year slider', start=2001, end=2023, value=2005)
year_slider.servable()
# # Display the slider
# year_slider

IntSlider(end=2023, name='Year slider', start=2001, value=2005)

In [8]:
# Radio Buttons for Currency 
yaxis_price = pn.widgets.RadioButtonGroup(
    name='Y axis',
    options=['price per KG(SDG)','price per KG(USD)',],
    button_type='success'
)

In [9]:
regions = ['South', 'East', 'Central', 'West', 'North']

price_pipeline = (
    idf[
        (idf.year <= year_slider) &
        (idf.region.isin(regions))
    ]
    .groupby(['region', 'year'])[yaxis_price].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True)
)


In [10]:
price_plot = price_pipeline.hvplot(x='year', by='region', y=yaxis_price, line_width=2, title='Price of food by region')

# Display the plot
price_plot

# (2) Table - Food prices over time by region

In [11]:
price_table = price_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width')
price_table

In [12]:
df['commodity'].unique()

array(['Sorghum (white)', 'Millet', 'Wheat', 'Sorghum',
       'Sorghum (food aid)'], dtype=object)

# (3) Bar chart with food prices by State  

In [13]:
df['state'].unique()

array(['Blue Nile', 'Gedaref', 'Khartoum', 'North Kordofan',
       'North Darfur', 'Northern', 'Eastern Darfur', 'Kassala', 'Red Sea',
       'South Darfur', 'West Darfur', 'White Nile', 'South Kordofan',
       'Central Darfur', 'West Kordofan', 'Al Gezira', 'Nile', 'Sennar'],
      dtype=object)

In [14]:

states= ['Blue Nile', 'Gedaref', 'Khartoum', 'North Kordofan','North Darfur', 'Northern', 'Eastern Darfur', 'Kassala', 'Red Sea','South Darfur', 'West Darfur', 'White Nile', 'South Kordofan','Central Darfur', 'West Kordofan', 'Al Gezira', 'Nile', 'Sennar']

state_bar_pipeline = (
    idf[
        (idf.year == year_slider) &
        (idf.state.isin(states))
    ]
    .groupby(['year', 'state'])[yaxis_price].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

In [15]:
state_bar_plot = state_bar_pipeline.hvplot(kind='bar', 
                                                     x='state', 
                                                     y=yaxis_price, 
                                                     title='Food prices by state',
                                          rot=45)

state_bar_plot

(4) Bar Chart with Food prices by commodity 

In [16]:
commodities =['Sorghum (white)', 'Millet', 'Wheat', 'Sorghum','Sorghum (food aid)']

commodity_bar_pipeline = (
    idf[
        (idf.year == year_slider) &
        (idf.commodity.isin(commodities))
    ]
    .groupby(['year','commodity'])[yaxis_price].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

In [17]:
commodity_bar_plot = commodity_bar_pipeline.hvplot(kind='bar', 
                                                     x='commodity', 
                                                     y=yaxis_price, 
                                                     title='Food prices by commodity',
                                          rot=45)
commodity_bar_plot


In [18]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='Sudan Food Prices', 
    sidebar=[pn.pane.Markdown("# Food Inflation Over time"), 
             pn.pane.Markdown("#### Carbon dioxide emissions are the primary driver of global climate change. It’s widely recognised that to avoid the worst impacts of climate change, the world needs to urgently reduce emissions. But, how this responsibility is shared between regions, countries, and individuals has been an endless point of contention in international discussions."), 
             pn.pane.PNG('sudan-map.png', sizing_mode='scale_both')],
    main=[
        pn.Row(price_plot, height=400),
                                    pn.GridBox(
                                    price_table,
                                    commodity_bar_plot,
                                    ncols=2,
                                    align='start',
                                    sizing_mode='scale_width'
                                    )
                            ]
                        )

template.show()
        
        
        

Launching server at http://localhost:58623


In [19]:
sidebar = [
    pn.pane.Markdown("# Food Inflation Over time"),
    pn.pane.Markdown("#### Carbon dioxide emissions are the primary driver of global climate change. It’s widely recognised that to avoid the worst impacts of climate change, the world needs to urgently reduce emissions. But, how this responsibility is shared between regions, countries, and individuals has been an endless point of contention in international discussions."),pn.pane.PNG('sudan-map.png', sizing_mode='scale_both'),
    pn.pane.Markdown("## Settings"),
    year_slider
]

# Remove yaxis_price from the main section and place it as the top element
main = [
    price_plot,
    pn.Row(price_table, commodity_bar_plot, ncols=2, align='start', sizing_mode='scale_width')
]

# Layout using Template
template = pn.template.FastListTemplate(
    title='Sudan Food Prices',
    sidebar=sidebar,
    main=main
)

template.add_panel('Price Widget', yaxis_price, area='header')

template.show()

AttributeError: 'FastListTemplate' object has no attribute 'add_panel'

In [141]:
import os
current_directory = os.getcwd()

# Print the current working directory
print("Current Directory:", current_directory)

Current Directory: C:\Users\hanee\myenv
